In [1]:
import numpy as np
import IPython.display as ipd
import datasets
import librosa

from sklearn import preprocessing
from sklearn import neighbors
from sklearn.model_selection import KFold

In [2]:
train_data = datasets.load_dataset(samples_dir='data/training_samples', feature='mfcc', sample_dur=100)
print('Created dataset')

Created dataset


In [3]:
def data_tuple(data, standardize=True, normalize=True):
    filenames = np.array([a[0] for a in data])
    features = np.array([a[1] for a in data])
    labels = np.array([a[2] for a in data])
    
    if standardize:
        features = (features - features.mean(0)) / features.std(0)
        
    if normalize:
        features = preprocessing.normalize(features)

    return (filenames, features, labels)

In [4]:
knn = neighbors.KNeighborsClassifier(n_neighbors=5)

# k-fold cross validation
cv_k = 10
kfold = KFold(cv_k, shuffle=True, random_state=1)

filenames, features, labels = data_tuple(train_data)

scores = []

for train, test in kfold.split(features):
    knn.fit(features[train], labels[train])
    scores.append(knn.score(features[test], labels[test]))

scores = np.array(scores)
print('k-NN performance: {:.2%} +/- {:.2%}'.format(scores.mean(), scores.std()))
print('Testing accuracy: {:.2%}: '.format(knn.score(features, labels)))

k-NN performance: 70.00% +/- 40.00%
Testing accuracy: 75.00%: 


In [ ]:
test_data = datasets.load_dataset(samples_dir='data/test_samples', feature='mfcc', sample_dur=32)

In [ ]:
filenames, features, labels = data_tuple(test_data)
print('Test score: ', knn.score(features, labels))

In [ ]:
# Predict random sample
idx = np.random.choice(len(filenames), 1)[0]
file = f'data/test_samples/{filenames[idx]}'
y, sr = librosa.load(file)
print(f'file: {file}')
ipd.display(ipd.Audio(file, rate=sr))
print('prediction: ', knn.predict([features[idx]]))